In [1]:
import os

In [2]:
%pwd

'c:\\M.sc Data Science clg\\DS SEM 3\\Medical_Image_Classification_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\M.sc Data Science clg\\DS SEM 3\\Medical_Image_Classification_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-22 23:27:40,209: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-22 23:27:40,252: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-22 23:27:40,257: INFO: common: created directory at: artifacts]
[2023-11-22 23:27:40,261: INFO: common: created directory at: artifacts/data_ingestion]


[2023-11-22 23:40:10,502: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 747199474
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "b9570e61b0b631baef2cf93c150b36be7278b1753ddbe8b2dfc35bdfb6c45b4e"
Last-Modified: Wed, 22 Nov 2023 17:55:34 GMT
Strict-Transport-Security: max-age=31557600
timing-allow-origin: https://github.com
X-Content-Type-Options: nosniff
X-Frame-Options: deny
x-github-tenant: 
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9C3A:3E24:9799B9:B39EC5:655E4093
Accept-Ranges: bytes
Date: Wed, 22 Nov 2023 17:57:44 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10243-MAA
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1700675865.525444,VS0,VE1
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 0493abd2b371e877c4e50adaa62179c3f31a83df
Expires: Wed, 22 Nov 2023 18:02:44 GMT
Source-Age: 130
Vary: Authorization,Acc